In [1]:
import folium

In [71]:
help(folium.GeoJson)

Help on class GeoJson in module folium.features:

class GeoJson(folium.map.Layer)
 |  GeoJson(data, style_function=None, highlight_function=None, name=None, overlay=True, control=True, show=True, smooth_factor=None, tooltip=None, embed=True, popup=None, zoom_on_click=False, marker=None)
 |  
 |  Creates a GeoJson object for plotting into a Map.
 |  
 |  Parameters
 |  ----------
 |  data: file, dict or str.
 |      The GeoJSON data you want to plot.
 |      * If file, then data will be read in the file and fully
 |      embedded in Leaflet's JavaScript.
 |      * If dict, then data will be converted to JSON and embedded
 |      in the JavaScript.
 |      * If str, then data will be passed to the JavaScript as-is.
 |      * If `__geo_interface__` is available, the `__geo_interface__`
 |      dictionary will be serialized to JSON and
 |      reprojected if `to_crs` is available.
 |  style_function: function, default None
 |      Function mapping a GeoJson Feature to a style dict.
 |  hig

In [37]:
import pandas
import os
dir = os.getcwd()
input_dir = os.path.join(dir, "InputFiles")
input_file_path = os.path.join(input_dir, "Volcanoes.csv")
df = pandas.read_csv(input_file_path)
df

,VOLCANX020,NUMBER,NAME,LOCATION,STATUS,ELEV,TYPE,TIMEFRAME,LAT,LON
0,509.0,1201-01=,Baker,US-Washington,Historical,3285.0,Stratovolcanoes,D3,48.776798,-121.810997
1,511.0,1201-02-,Glacier Peak,US-Washington,Tephrochronology,3213.0,Stratovolcano,D4,48.111801,-121.111000
2,513.0,1201-03-,Rainier,US-Washington,Dendrochronology,4392.0,Stratovolcano,D3,46.869801,-121.751000
3,515.0,1201-05-,St. Helens,US-Washington,Historical,2549.0,Stratovolcano,D1,46.199799,-122.181000
4,516.0,1201-04-,Adams,US-Washington,Tephrochronology,3742.0,Stratovolcano,D6,46.205799,-121.490997
...,...,...,...,...,...,...,...,...,...,...
57,580.0,1210-03-,Valles Caldera,US-New Mexico,Pleistocene-Fumarolic,3430.0,Caldera,Q,35.869999,-106.570999
58,581.0,1209-02-,Sunset Crater,US-Arizona,Dendrochronology,2447.0,Cinder cone,D6,35.369999,-111.501000
59,584.0,1203-20-,Amboy,US-California,Holocene,288.0,Cinder cone,U,34.549999,-115.780998
60,586.0,1210-02-,Zuni-Bandera,US-New Mexico,Anthropology,2550.0,Volcanic field,D7,34.799999,-108.001000


In [ ]:
def split_range(min_range, max_range, split_count):
    actual_range = max_range - min_range
    chunk = actual_range / split_count
    data= min_range
    range_list = [data]
    while(data + chunk <= max_range):
        data += chunk
        range_list.append(data)
    return range_list[1:]

In [56]:
def get_color(elevation, colors, ranges):
    for i in range(len(ranges)):
        if elevation <= ranges[i]:
            return colors[i]

In [85]:
html = """<h4>Volcano information:</h4>
Height: %s m
"""
map = folium.Map(location=[df["LAT"][0], df["LON"][0]], tiles="Stamen Terrain", zoom_start=5)
fgv = folium.FeatureGroup(name="Fgv")
fgp = folium.FeatureGroup(name="Fgp")

ranges = split_range(min(df["ELEV"]), max(df["ELEV"]), 3)
colors = ["green", "blue", "red"]

for i in range(len(df)):   
    iframe = folium.IFrame(html=html % str(df["ELEV"][i]), width=200, height=100)
    fgv.add_child(folium.CircleMarker(location=[df["LAT"][i], df["LON"][i]], radius = 10, popup=folium.Popup(iframe), \
                fill_color=get_color(df["ELEV"][i], colors, ranges), color = "grey", fill_opacity=0.7))
 
# Add GeoJson child
data = open(os.path.join(input_dir, "world.json"), "r", encoding='utf-8-sig').read()
fgp.add_child(folium.GeoJson(data, style_function=lambda x:{'fillColor':'yellow' if x["properties"]["POP2005"] < 10000000
                                                           else 'blue' if x["properties"]["POP2005"] < 50000000 else 'red'}))


map.add_child(fgv)
map.add_child(fgp)
map.add_child(folium.LayerControl())
map.save("Map1.html")